In [16]:
import os
!pip install kaggle --upgrade


Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


In [0]:
os.environ['KAGGLE_USERNAME'] = "dileepsn"
os.environ['KAGGLE_KEY'] = "5b6a34d0465caab35e9ae3a1da050525"


In [20]:
!kaggle competitions download -c plant-seedlings-classification

  0% 0.00/5.13k [00:00<?, ?B/s]
100% 5.13k/5.13k [00:00<00:00, 1.98MB/s]
 95% 82.0M/86.0M [00:00<00:00, 157MB/s]
100% 86.0M/86.0M [00:00<00:00, 159MB/s]
100% 1.59G/1.60G [00:17<00:00, 126MB/s]
100% 1.60G/1.60G [00:17<00:00, 98.6MB/s]


In [0]:
from zipfile import ZipFile

with ZipFile('train.zip','r') as f:
  f.extractall()

In [0]:
with ZipFile('test.zip','r') as f:
  f.extractall()

1.Read the images and generate the train and test dataset


In [24]:
x_train =[]
y_train = []
x_test = []
os.listdir('train')

['Common wheat',
 'Fat Hen',
 'Loose Silky-bent',
 'Maize',
 'Charlock',
 'Sugar beet',
 'Black-grass',
 'Shepherds Purse',
 'Common Chickweed',
 'Cleavers',
 'Scentless Mayweed',
 'Small-flowered Cranesbill']

In [0]:
import cv2
for i in os.listdir('train'):
  for j in os.listdir('train/'+ i):
    temp = cv2.imread('train/'+ i +'/'+j)
    temp = cv2.resize(temp,(128,128))
    x_train.append(temp)
    y_train.append(i)


In [29]:
y_train[-5:]

['Small-flowered Cranesbill',
 'Small-flowered Cranesbill',
 'Small-flowered Cranesbill',
 'Small-flowered Cranesbill',
 'Small-flowered Cranesbill']

In [30]:
len(x_train)

4750

In [27]:
!pwd

/content/train


In [0]:
for i in os.listdir('test'):
  tmp2 = cv2.imread('test/'+i)
  tmp2 = cv2.resize(tmp2,(128,128))
  x_test.append(tmp2)


In [32]:
len(x_test)

794

In [33]:
print(x_train[0].shape)

print(x_test[0].shape)

import numpy as np

x_train = np.array(x_train)
x_test = np.array(x_test)

print(x_train[0].shape)

print(x_test[0].shape)


(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)


In [0]:
import pandas as pd
y_train = pd.get_dummies(y_train)

In [35]:
y_train

,Black-grass,Charlock,Cleavers,Common Chickweed,Common wheat,Fat Hen,Loose Silky-bent,Maize,Scentless Mayweed,Shepherds Purse,Small-flowered Cranesbill,Sugar beet
0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4745,0,0,0,0,0,0,0,0,0,0,1,0
4746,0,0,0,0,0,0,0,0,0,0,1,0
4747,0,0,0,0,0,0,0,0,0,0,1,0
4748,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
y_train = np.array(y_train)

2. Divide the data set into Train and validation data sets


In [37]:
from sklearn.model_selection import train_test_split
x_train2,x_test2,y_train2,y_test2 = train_test_split(x_train,y_train,test_size=0.2,random_state=123)
print (x_train2.shape)
print (x_test2.shape)

(3800, 128, 128, 3)
(950, 128, 128, 3)


In [0]:
#normalizing the data
x_train2 = x_train2/255
x_test2 = x_test2/255

3. Initialize & build the model 
4. Optimize the model 
5. Predict the accuracy for both train and validation data (7 points)


In [39]:
from keras.models import Sequential

Using TensorFlow backend.


In [40]:
model = Sequential()

In [41]:
from keras.layers import BatchNormalization
model.add(BatchNormalization(input_shape=(128,128,3)))

In [0]:
from keras.layers import Convolution2D, Dropout, Dense ,MaxPool2D ,Flatten

In [43]:
model.add(Convolution2D(32,(3,3),activation='relu',input_shape = (128, 128, 3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Convolution2D(64,kernel_size=4,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(0.3))

model.add(Convolution2D(128,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(0.3))

model.add(Convolution2D(128,kernel_size=2,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
#Full connected layers
model.add(Dense(128,activation='relu'))

model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 12, activation = 'softmax')) 


In [45]:
#optimizers

from keras.optimizers import adam
custom_optimizer = adam(lr=0.001)
model.compile(optimizer=custom_optimizer,metrics=['accuracy'],loss='categorical_crossentropy')

In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 128)      

In [48]:
model.fit(x_train2,y_train2,epochs=30,batch_size=48,validation_data=(x_test2,y_test2),initial_epoch=0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 3800 samples, validate on 950 samples
Epoch 1/30
3800/3800 [==============================] - 193s 51ms/step - loss: 2.3303 - acc: 0.1766 - val_loss: 2.0048 - val_acc: 0.2832
Epoch 2/30
3800/3800 [==============================] - 190s 50ms/step - loss: 2.0151 - acc: 0.3024 - val_loss: 1.8916 - val_acc: 0.3905
Epoch 3/30
3800/3800 [==============================] - 189s 50ms/step - loss: 1.7027 - acc: 0.4255 - val_loss: 1.3528 - val_acc: 0.5653
Epoch 4/30
3800/3800 [==============================] - 186s 49ms/step - loss: 1.4503 - acc: 0.5084 - val_loss: 1.1456 - val_acc: 0.6411
Epoch 5/30
3800/3800 [==============================] - 179s 47ms/step - loss: 1.2904 - acc: 0.5726 - val_loss: 1.1012 - val_acc: 0.6537
Epoch 6/30
3800/3800 [==============================] - 174s 46ms/step - loss: 1.1521 - acc: 0.6179 - val_loss: 0.9865 - val_acc: 0.6811
Epoch 7/30
3800/3800 [=============

In [50]:
#Checking for 5 more iterations
model.fit(x_train2,y_train2,epochs=5,batch_size=48,validation_data=(x_test2,y_test2))


Train on 3800 samples, validate on 950 samples
Epoch 1/5
3800/3800 [==============================] - 171s 45ms/step - loss: 0.2202 - acc: 0.9171 - val_loss: 0.7938 - val_acc: 0.8137
Epoch 2/5
3800/3800 [==============================] - 171s 45ms/step - loss: 0.2333 - acc: 0.9179 - val_loss: 0.6803 - val_acc: 0.8432
Epoch 3/5
3800/3800 [==============================] - 171s 45ms/step - loss: 0.2010 - acc: 0.9271 - val_loss: 0.7695 - val_acc: 0.8305
Epoch 4/5
3800/3800 [==============================] - 173s 45ms/step - loss: 0.1853 - acc: 0.9308 - val_loss: 0.7228 - val_acc: 0.8421
Epoch 5/5
3800/3800 [==============================] - 173s 45ms/step - loss: 0.2009 - acc: 0.9308 - val_loss: 0.7942 - val_acc: 0.8263


In [0]:
model.save('/content/drive/My Drive/Colab Notebooks/Notebooks/MY_PlantClassification_model.h5')
model.save_weights('/content/drive/My Drive/Colab Notebooks/Notebooks/MY_Plant_Classification_weights.h5')

In [52]:
#Testing the model on test set
score = model.evaluate(x_train2, y_train2)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

3800/3800 [==============================] - 47s 12ms/step
Train loss: 0.09483729518361782
Train accuracy: 0.9689473682955692


In [51]:
#Testing the model on test set
score = model.evaluate(x_test2, y_test2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

950/950 [==============================] - 11s 12ms/step
Test loss: 0.7942198094568754
Test accuracy: 0.8263157893482007
